In [4]:
# import python libraries
from netCDF4 import Dataset, num2date
from pathlib import Path

## Run these two cells below only once

In [ ]:
# Create a key ascii file
!echo "url: https://cds.climate.copernicus.eu/api/v2" > ~/.cdsapirc 

# Replace UID with your user ID and KEY with your API key
!echo "key: UID:KEY" >> ~/.cdsapirc

In [6]:
# install era5cli
!pip install era5cli

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


## Settings

In [7]:
# path to forcing data
forcing_path  = "/data/shared/EcoExtreML/STEMMUS_SCOPEv1.0.0/input/Plumber2_data/"

## Test run using --dryrun

In [30]:
forcing_name = "AU-ASM_2011-2017_OzFlux_Met.nc"
station_name = "AU-ASM"
# read data
filename = Path(forcing_path, forcing_name) # care for windows/unix paths
nc_fid = Dataset(filename, mode='r')

nctime = nc_fid.variables['time']
ncdate = num2date(nctime[:],units= nctime.units, calendar=nctime.calendar)

startyear = ncdate[0].year
endyear = startyear

startmonth = ncdate[0].month
startday = ncdate[0].day
starthour = ncdate[0].hour

lat = float(nc_fid.variables['latitude'][:].flatten()[0])
lon = float(nc_fid.variables['longitude'][:].flatten()[0])

In [31]:
print(startyear, endyear, startmonth, startday, starthour, lat, lon)

2011 2011 1 1 0 -22.283000946044922 133.24899291992188


In [32]:
lat_max = lat + 0.001
lat_min = lat - 0.001
lon_max = lon + 0.001
lon_min = lon - 0.001
print(lat_max, lon_min, lat_min, lon_max)

-22.28200094604492 133.24799291992187 -22.284000946044923 133.24999291992188


In [38]:
timestamp = ncdate[0].strftime('%Y%m%d_%H')
filename = f"{station_name}_{timestamp}"

In [40]:
!era5cli hourly --variables soil_temperature_level_1 volumetric_soil_water_layer_1 skin_temperature --startyear {startyear} --endyear {endyear} --land --area {lat_max} {lon_min} {lat_min} {lon_max} --months {startmonth} --days {startday} --hours {starthour} --outputprefix {filename} --dryrun --merge

NB: coordinates [-22.28200094604492, 133.24799291992187, -22.284000946044923, 133.24999291992188] rounded down to two decimals.
NB: coordinates [-22.28200094604492, 133.24799291992187, -22.284000946044923, 133.24999291992188] rounded down to two decimals.

NB: coordinates [-22.28200094604492, 133.24799291992187, -22.284000946044923, 133.24999291992188] rounded down to two decimals.
reanalysis-era5-land

 reanalysis-era5-landreanalysis-era5-land{'variable': 'soil_temperature_level_1', 'year': [2011], 'month': ['01'], 'time': ['00:00'], 'format': 'netcdf', 'area': [-22.28, 133.25, -22.28, 133.25], 'day': ['01']}   {'variable': 'volumetric_soil_water_layer_1', 'year': [2011], 'month': ['01'], 'time': ['00:00'], 'format': 'netcdf', 'area': [-22.28, 133.25, -22.28, 133.25], 'day': ['01']}{'variable': 'skin_temperature', 'year': [2011], 'month': ['01'], 'time': ['00:00'], 'format': 'netcdf', 'area': [-22.28, 133.25, -22.28, 133.25], 'day': ['01']}AU-ASM_20110101_00-land_soil_temperature_leve

## Download data for 170

In [ ]:
forcing_filenames_list = ["AU-ASM_2011-2017_OzFlux_Met.nc", "AR-SLu_2010-2010_FLUXNET2015_Met.nc"]

## if you want to download all 170 sites, change False to True
fullrun = False
if fullrun:
    forcing_filenames_list = [file.name for file in Path(forcing_path).iterdir()]
    
for forcing_name in forcing_filenames_list:
    # read data
    filename = Path(forcing_path, forcing_name) # care for windows/unix paths
    nc_fid = Dataset(filename, mode='r')

    nctime = nc_fid.variables['time']
    ncdate = num2date(nctime[:],units= nctime.units, calendar=nctime.calendar)
    
    startyear = ncdate[0].year
    endyear = startyear

    startmonth = ncdate[0].month
    startday = ncdate[0].day
    starthour = ncdate[0].hour

    lat = float(nc_fid.variables['latitude'][:].flatten()[0])
    lon = float(nc_fid.variables['longitude'][:].flatten()[0])
    
    # create area
    lat_max = lat + 0.001
    lat_min = lat - 0.001
    lon_max = lon + 0.001
    lon_min = lon - 0.001
    
    # create prefix for file names
    timestamp = ncdate[0].strftime('%Y%m%d_%H')
    filename = f"{station_name}_{timestamp}"
    
    # Download data
    station_name = ncfile.split('_')[0]
    !era5cli hourly --variables soil_temperature_level_1 volumetric_soil_water_layer_1 --startyear {startyear} --endyear {endyear} --land --area {lat_max} {lon_min} {lat_min} {lon_max} --outputprefix {filename} --merge
    